In [7]:
#!pip install requests kafka-python

In [1]:
import requests
import json
import time
import csv
import os
from kafka import KafkaProducer

# Fungsi untuk inisialisasi Kafka Producer
def inisialisasi_kafka_producer():
    try:
        producer = KafkaProducer(
            bootstrap_servers='localhost:9092',  
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        print("Kafka Producer berhasil diinisialisasi.")
        return producer
    except Exception as e:
        print(f"Gagal menginisialisasi Kafka Producer: {e}")
        return None

# API Key dan URL untuk API Cuaca Realtime WeatherAPI
api_key = '81c93728c6cf49f496052543242005'
base_url = "http://api.weatherapi.com/v1/current.json"

# Fungsi untuk mengambil data cuaca dari WeatherAPI
def ambil_data_cuaca(lokasi):
    try:
        url = f"{base_url}?key={api_key}&q={lokasi}"
        response = requests.get(url)
        response.raise_for_status()  # Memunculkan error untuk status kode yang buruk
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"Gagal mengambil data dari WeatherAPI: {e}")
        return None

# Fungsi untuk mengirim data ke Kafka dan menyimpan ke CSV
def kirim_ke_kafka(producer, data, topik, csv_writer):
    if producer and data:
        try:
            producer.send(topik, data)
            print(f"Terkirim ke Kafka: {data}")
            
            # Simpan data ke CSV
            location = data['location']
            current = data['current']
            csv_writer.writerow([
                location['name'],
                location['region'],
                location['country'],
                location['lat'],
                location['lon'],
                location['tz_id'],
                location['localtime'],
                current['last_updated'],
                current['temp_c'],
                current['temp_f'],
                current['is_day'],
                current['condition']['text'],
                current['condition']['code'],
                current['wind_mph'],
                current['wind_kph'],
                current['wind_degree'],
                current['pressure_in'],
                current['precip_in'],
                current['humidity'],
                current['cloud'],
                current['feelslike_c'],
                current['uv'],
                current['gust_mph'],
                current['gust_kph']
            ])
        except Exception as e:
            print(f"Gagal mengirim data ke Kafka: {e}")

# Loop utama untuk polling data secara berkala
def main():
    topik = 'cuaca-realtime'  # Ganti dengan nama topik Kafka Anda
    lokasi_surabaya = 'Surabaya'  # Lokasi Surabaya
    lokasi_malang = 'Malang'  # Lokasi Malang
    max_iterasi = 500  # Tentukan jumlah iterasi maksimum sebelum berhenti
    interval_polling = 60  # Interval polling dalam detik
    iterasi = 0

    producer = inisialisasi_kafka_producer()
    if not producer:
        print("Kafka Producer tidak dapat diinisialisasi. Keluar.")
        return
    
    # Nama file CSV
    csv_file = 'cuaca_realtime_sby_mlg.csv'
    
    # Periksa apakah file CSV sudah ada
    file_exists = os.path.isfile(csv_file)

    # Buka file CSV untuk ditulis
    with open(csv_file, mode='a', newline='') as file:
        csv_writer = csv.writer(file)
        
        # Tulis header jika file baru
        if not file_exists:
            csv_writer.writerow([
                'name', 'region', 'country', 'lat', 'lon', 'tz_id', 'localtime', 
                'last_updated', 'temp_c', 'temp_f', 'is_day', 'condition', 'code', 'wind_mph', 'wind_kph',
                'wind_degree', 'pressure_in', 'precip_in', 'humidity', 'cloud', 'feelslike_c', 'uv', 'gust_mph', 'gust_kph'
            ])
        
        while iterasi < max_iterasi:
            start_time = time.time()  # Waktu awal sebelum polling
            
            # Ambil data cuaca untuk Surabaya
            data_surabaya = ambil_data_cuaca(lokasi_surabaya)
            if data_surabaya:
                kirim_ke_kafka(producer, data_surabaya, topik, csv_writer)
            
            # Ambil data cuaca untuk Malang
            data_malang = ambil_data_cuaca(lokasi_malang)
            if data_malang:
                kirim_ke_kafka(producer, data_malang, topik, csv_writer)
            
            # Hitung waktu yang dibutuhkan untuk polling
            elapsed_time = time.time() - start_time
            sleep_time = max(0, interval_polling - elapsed_time)  # Pastikan sleep_time tidak negatif
            time.sleep(sleep_time)
            
            iterasi += 1

    print("Polling selesai. Menutup Kafka Producer.")
    producer.close()

if __name__ == "__main__":
    main()


Kafka Producer berhasil diinisialisasi.
Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716255932, 'localtime': '2024-05-21 8:45'}, 'current': {'last_updated_epoch': 1716255900, 'last_updated': '2024-05-21 08:45', 'temp_c': 30.0, 'temp_f': 86.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 100, 'wind_dir': 'E', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 25, 'feelslike_c': 37.6, 'feelslike_f': 99.7, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 8.0, 'gust_mph': 10.1, 'gust_kph': 16.2}}
Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716255872, 'localtime': '202

Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716256319, 'localtime': '2024-05-21 8:51'}, 'current': {'last_updated_epoch': 1716255900, 'last_updated': '2024-05-21 08:45', 'temp_c': 19.7, 'temp_f': 67.4, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 176, 'wind_dir': 'S', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.02, 'precip_in': 0.0, 'humidity': 85, 'cloud': 85, 'feelslike_c': 19.7, 'feelslike_f': 67.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.0, 'gust_mph': 4.3, 'gust_kph': 6.8}}
Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716255932, 'localtime': '2024-05-21 8:45'}, 'current': {'last_u

Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716256303, 'localtime': '2024-05-21 8:51'}, 'current': {'last_updated_epoch': 1716255900, 'last_updated': '2024-05-21 08:45', 'temp_c': 30.0, 'temp_f': 86.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 100, 'wind_dir': 'E', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 25, 'feelslike_c': 37.6, 'feelslike_f': 99.7, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 8.0, 'gust_mph': 10.1, 'gust_kph': 16.2}}
Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716256558, 'localtime': '2024-05-21 8:55'}, 'current': {'last_update

Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716256558, 'localtime': '2024-05-21 8:55'}, 'current': {'last_updated_epoch': 1716255900, 'last_updated': '2024-05-21 08:45', 'temp_c': 19.7, 'temp_f': 67.4, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 176, 'wind_dir': 'S', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.02, 'precip_in': 0.0, 'humidity': 85, 'cloud': 85, 'feelslike_c': 19.7, 'feelslike_f': 67.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.0, 'gust_mph': 4.3, 'gust_kph': 6.8}}
Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716256915, 'localtime': '2024-05-21 9:01'}, 'current': {'last_u

Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716257336, 'localtime': '2024-05-21 9:08'}, 'current': {'last_updated_epoch': 1716256800, 'last_updated': '2024-05-21 09:00', 'temp_c': 30.0, 'temp_f': 86.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 100, 'wind_dir': 'E', 'pressure_mb': 1010.0, 'pressure_in': 29.83, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 70, 'cloud': 25, 'feelslike_c': 35.8, 'feelslike_f': 96.5, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 8.0, 'gust_mph': 10.1, 'gust_kph': 16.2}}
Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716257277, 'localtime': '2024-05-21 9:07'}, 'current': {'last_update

...

Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716284339, 'localtime': '2024-05-21 16:38'}, 'current': {'last_updated_epoch': 1716283800, 'last_updated': '2024-05-21 16:30', 'temp_c': 32.0, 'temp_f': 89.6, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 12.5, 'wind_kph': 20.2, 'wind_degree': 60, 'wind_dir': 'ENE', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 63, 'cloud': 25, 'feelslike_c': 36.9, 'feelslike_f': 98.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 9.0, 'gust_mph': 17.0, 'gust_kph': 27.4}}
Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716284340, 'localtime': '2024-05-21 16:39'}, 'current': {'last_

Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716284760, 'localtime': '2024-05-21 16:46'}, 'current': {'last_updated_epoch': 1716284700, 'last_updated': '2024-05-21 16:45', 'temp_c': 21.5, 'temp_f': 70.7, 'is_day': 1, 'condition': {'text': 'Light rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png', 'code': 1240}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 151, 'wind_dir': 'SSE', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.16, 'precip_in': 0.01, 'humidity': 84, 'cloud': 55, 'feelslike_c': 21.5, 'feelslike_f': 70.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 6.7, 'gust_kph': 10.8}}
Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716284760, 'localtime': '2024-05-21 16:46'}, 'current': {'la

Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716284999, 'localtime': '2024-05-21 16:49'}, 'current': {'last_updated_epoch': 1716284700, 'last_updated': '2024-05-21 16:45', 'temp_c': 31.0, 'temp_f': 87.8, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 80, 'wind_dir': 'E', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 25, 'feelslike_c': 34.8, 'feelslike_f': 94.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 9.0, 'gust_mph': 16.3, 'gust_kph': 26.3}}
Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716285000, 'localtime': '2024-05-21 16:50'}, 'current': {'last_up

Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716285000, 'localtime': '2024-05-21 16:50'}, 'current': {'last_updated_epoch': 1716284700, 'last_updated': '2024-05-21 16:45', 'temp_c': 21.5, 'temp_f': 70.7, 'is_day': 1, 'condition': {'text': 'Light rain shower', 'icon': '//cdn.weatherapi.com/weather/64x64/day/353.png', 'code': 1240}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 151, 'wind_dir': 'SSE', 'pressure_mb': 1009.0, 'pressure_in': 29.8, 'precip_mm': 0.16, 'precip_in': 0.01, 'humidity': 84, 'cloud': 55, 'feelslike_c': 21.5, 'feelslike_f': 70.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 6.0, 'gust_mph': 6.7, 'gust_kph': 10.8}}
Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716285480, 'localtime': '2024-05-21 16:58'}, 'current': {'la

Terkirim ke Kafka: {'location': {'name': 'Surabaya', 'region': 'East Java', 'country': 'Indonesia', 'lat': -7.25, 'lon': 112.75, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716285720, 'localtime': '2024-05-21 17:02'}, 'current': {'last_updated_epoch': 1716285600, 'last_updated': '2024-05-21 17:00', 'temp_c': 31.0, 'temp_f': 87.8, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 80, 'wind_dir': 'E', 'pressure_mb': 1006.0, 'pressure_in': 29.71, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 66, 'cloud': 25, 'feelslike_c': 35.3, 'feelslike_f': 95.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 8.0, 'gust_mph': 16.3, 'gust_kph': 26.3}}
Terkirim ke Kafka: {'location': {'name': 'Malang', 'region': 'East Java', 'country': 'Indonesia', 'lat': -8.0, 'lon': 114.14, 'tz_id': 'Asia/Jakarta', 'localtime_epoch': 1716285721, 'localtime': '2024-05-21 17:02'}, 'current': {'last_u